<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-CNN-Projects/blob/master/Flower-Images-MobileNet-Edge-Optimization/edge_optimization_flower_images_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import tensorflow as tf
import numpy as np
import os
import pathlib


# Load Data

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir) / 'flower_photos'

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


# Loading Datasets

In [3]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset="training", seed=123,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset="validation", seed=123,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE
)

def preprocess(image, label):
    return tf.keras.applications.mobilenet_v2.preprocess_input(image), label

train_ds = train_ds.map(preprocess).cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().prefetch(tf.data.AUTOTUNE)


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


# Load `MobileNetV2`

In [4]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet'
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, epochs=5, validation_data=val_ds, verbose=1)

model.save("flower_model_float.keras")
float_size = os.path.getsize("flower_model_float.keras") / (1024*1024)
print(f"Base Model Size: {float_size:.2f} MB")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 71s 532ms/step - accuracy: 0.5810 - loss: 1.0876 - val_accuracy: 0.8406 - val_loss: 0.4392
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.8536 - loss: 0.4230 - val_accuracy: 0.8665 - val_loss: 0.3645
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8851 - loss: 0.3426 - val_accuracy: 0.8896 - val_loss: 0.3368
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9029 - loss: 0.2812 - val_accuracy: 0.8869 - val_loss: 0.3208
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.9164 - loss: 0.2527 - val_accuracy: 0.8937 - val_loss: 0.3085
Base Model Size: 9.25 MB


# Representative Dataset

In [5]:
def representative_data_gen():
    for input_value, _ in train_ds.unbatch().batch(1).take(100):
        yield [input_value]

print("Representative dataset generator ready.")

Representative dataset generator ready.


In [6]:
# Full Integer TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

QUANT_FILE = "flower_model_full_int8.tflite"
with open(QUANT_FILE, 'wb') as f:
    f.write(tflite_model_quant)

quant_size = os.path.getsize(QUANT_FILE) / (1024*1024)
print(f"Full Integer Model Saved: {quant_size:.2f} MB")

Saved artifact at '/tmp/tmp27yojwej'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  135934191807632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191808016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191810320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191809936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191809360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191807824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191810512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191811280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191810896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135934191808208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1359341918

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Full Integer Model Saved: 2.59 MB


# Evaluation

In [7]:
def evaluate_int8_model(model_path, dataset):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]


    input_scale, input_zero_point = input_details["quantization"]

    print(f"Model Input Type: {input_details['dtype']}")
    print(f"Input Scale: {input_scale}, Zero Point: {input_zero_point}")

    correct = 0
    total = 0

    print("Running inference (converting float images to int8)...")


    for image_float, label in dataset.unbatch().take(200):

        image_float = tf.expand_dims(image_float, axis=0).numpy()
        image_int8 = (image_float / input_scale) + input_zero_point
        image_int8 = np.clip(image_int8, -128, 127).astype(np.int8)

        interpreter.set_tensor(input_details["index"], image_int8)
        interpreter.invoke()

        output_int8 = interpreter.get_tensor(output_details["index"])

        prediction = np.argmax(output_int8[0])

        if prediction == label.numpy():
            correct += 1
        total += 1

    return correct / total

acc_int8 = evaluate_int8_model(QUANT_FILE, val_ds)

Model Input Type: <class 'numpy.int8'>
Input Scale: 0.007843137718737125, Zero Point: -1
Running inference (converting float images to int8)...


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [8]:
print("\n" + "="*50)
print("        FULL INTEGER QUANTIZATION RESULTS")
print("="*50)
print(f"{'Metric':<15} | {'Base (Float32)':<15} | {'Edge (Int8)':<15}")
print("-" * 50)
print(f"{'Size':<15} | {float_size:.2f} MB         | {quant_size:.2f} MB")
print(f"{'Accuracy':<15} | (Not calculated)  | {acc_int8:.2%}")
print("-" * 50)
print(f"Compression Ratio: {float_size / quant_size:.1f}x smaller")
print("Hardware Compatibility: CPU, Edge TPU, DSP, Microcontrollers")
print("="*50)


        FULL INTEGER QUANTIZATION RESULTS
Metric          | Base (Float32)  | Edge (Int8)    
--------------------------------------------------
Size            | 9.25 MB         | 2.59 MB
Accuracy        | (Not calculated)  | 88.50%
--------------------------------------------------
Compression Ratio: 3.6x smaller
Hardware Compatibility: CPU, Edge TPU, DSP, Microcontrollers
